In [1]:
from pyspark.sql import SparkSession

#créer une session dans le master
#spark = SparkSession.builder \
#    .master("spark://172.20.53.96:7077") \
#    .appName("WDC-complete") \
#    .config("spark.executor.memory","28g") \
#    .config("spark.driver.memory","28g") \
#    .getOrCreate()
spark = SparkSession.builder.master("local").appName("WDC-complete").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 21:44:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/13 21:44:44 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/04/13 21:44:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/13 21:44:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/13 21:44:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/13 21:44:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql import functions as f

from pyspark.sql import Row
import re
from urllib.parse import urlparse


line='_:nb53a24408607424384c1357880ce1bc7xb1 <http://schema.org/value> "8,840 ft<> / 2,694 m<>" <https://peakery.com/dragontail-peak-washington/>   .'
quad_motif = re.compile(r'([^\s]+)\s([^\s]+)\s(.+)\s([^\s]+)\s+\.')

def parseQ(l, parts):
  result=quad_motif.match(l)
  #print(result.groups())
  sub=result.group(1).strip()
  pred=result.group(2).strip()
  if pred=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
      pred="isa:"+result.group(3).strip()
  domain=urlparse(result.group(4).strip().strip("<>")).netloc
  return Row(subject=sub,predicate=pred,prov=domain,hashdom=hash(domain)%parts)

print(parseQ(line, 10))

Row(subject='_:nb53a24408607424384c1357880ce1bc7xb1', predicate='<http://schema.org/value>', prov='peakery.com', hashdom=6)


In [8]:
def partCS(hashdom_val, hashdom_max, output=None):
    print(f"part {hashdom_val+1}/{hashdom_max} started");
    file_name = f"hashdom{hashdom_val+1}-{hashdom_max}"
    cset2 = spark.sql(f"select subject, concat_ws(' ',sort_array(collect_set(predicate))) as pset FROM Super where hashdom={hashdom_val} group by  subject ").cache()
    #cset2.show(truncate=200)
    #print(cset2.count())

    result2 = cset2.groupby("pset").agg(f.count(cset2.subject).alias('count'))
    result2.show(truncate=0)

    if(output is not None):
        print("Saving")
        result2.write.option("header",True) \
            .mode("overwrite") \
            .csv(f"s3a://test-out/{output}/{file_name}")
    
    # clear variables from memory
    cset2.unpersist()
    result2.unpersist()

    print(f"part {hashdom_val+1}/{hashdom_max} finished");
    
    del cset2, result2

def completeCS(input, parts, output=None):
    lines = spark.sparkContext.textFile(f"s3a://{input}/")
    
    sp=lines.map(lambda l: parseQ(l, parts)).toDF()

    sp = sp.withColumn("predicate", f.regexp_replace(f.col("predicate"), "([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?", ""))
    sp.show(truncate=0)

    sp.createOrReplaceTempView("Super")
    
    for i in range(parts):
        partCS(i, parts, output)
        
    print("Finished")

In [ ]:
# input : test or wdc
# no output = no save
completeCS("test", 10)